In [77]:
import os
import nltk
import spacy
import gensim
import pyLDAvis
import pyLDAvis.gensim_models 
import warnings
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
warnings.filterwarnings("ignore", category=DeprecationWarning)

torch.set_num_threads(4)

nlp = spacy.load('en_core_web_trf')
stop = stopwords.words('english')
post_data = pd.read_csv("/home/theerthala/Documents/repos/Crowdfunding-Social-Media-Drivers/Data/2022-07-22-09-31-10-IST-feed-download.csv")

#basic data cleaning
post_data.drop_duplicates(subset=['Message'], keep='last', inplace=True)
post_data.reset_index(inplace=True, drop=True)
post_data.columns = [x.lower().replace(' ','_') for x in post_data.columns.to_list()]
post_data.drop(['user_name', 'facebook_id', 'page_category', 'page_admin_top_country',
                'overperforming_score_(weighted__—__likes_1x_shares_1x_comments_1x_love_1x_wow_1x_haha_1x_sad_1x_angry_1x_care_1x_)'],
                 axis=1, inplace=True)
post_data.head()

,page_name,page_created,likes_at_posting,followers_at_posting,post_created,post_created_date,post_created_time,type,total_interactions,likes,...,url,message,link,final_link,image_text,link_text,description,sponsor_id,sponsor_name,sponsor_category
0,GoFundMe,10-02-2010 02:10,1839689,1942958.0,2022-07-22 01:22:32 IST,22-07-2022,01:22:32,Photo,21,9,...,https://www.facebook.com/299947508549/posts/10...,“I knew I had to do this with as many veterans...,https://www.facebook.com/gofundme/photos/a.101...,NaN,HONOR,NaN,NaN,NaN,NaN,NaN
1,Kickstarter,21-04-2009 16:42,1589842,1587991.0,2022-07-22 01:18:36 IST,22-07-2022,01:18:36,Link,3,1,...,https://www.facebook.com/73182029884/posts/101...,Artist and designer Janos Stone is creating a ...,https://www.kickstarter.com/projects/haus/toyh...,https://www.kickstarter.com/projects/haus/toyh...,NaN,ToyHaus: a uniquely beautiful mini-playhouse f...,"ToyHaus is a washable, popup mini-playhouse wi...",NaN,NaN,NaN
2,Kickstarter,21-04-2009 16:42,1589842,1587991.0,2022-07-21 18:52:39 IST,21-07-2022,18:52:39,Link,9,4,...,https://www.facebook.com/73182029884/posts/101...,The inaugural meeting of Kickstarter’s Communi...,https://www.kickstarter.com/blog/reporting-bac...,https://www.kickstarter.com/blog/reporting-bac...,NaN,Reporting Back from the First Meeting of the K...,"Earlier this month, I was honored to host the ...",NaN,NaN,NaN
3,GoFundMe,10-02-2010 02:10,1839047,1942443.0,2022-07-21 00:23:03 IST,21-07-2022,00:23:03,Native Video,163,42,...,https://www.facebook.com/299947508549/posts/10...,This 25-year-old man rushed into a burning hou...,https://www.facebook.com/peoplemag/videos/2043...,NaN,NaN,People,"""Nicholas Bostic's heroic actions saved lives,...",NaN,NaN,NaN
4,Kickstarter,21-04-2009 16:42,1589866,1588006.0,2022-07-20 02:38:36 IST,20-07-2022,02:38:36,Link,7,5,...,https://www.facebook.com/73182029884/posts/101...,An occult-themed JRPG with a fully integrated ...,https://www.kickstarter.com/projects/cherrymoc...,https://www.kickstarter.com/projects/cherrymoc...,NaN,EXIT VEIL: Occult & Tarot JRPG,A Dark-Psychedelic JRPG & fully integrated Tar...,NaN,NaN,NaN


In [78]:
post_data.columns

Index(['page_name', 'page_created', 'likes_at_posting', 'followers_at_posting',
       'post_created', 'post_created_date', 'post_created_time', 'type',
       'total_interactions', 'likes', 'comments', 'shares', 'love', 'wow',
       'haha', 'sad', 'angry', 'care', 'video_share_status', 'is_video_owner?',
       'post_views', 'total_views', 'total_views_for_all_crossposts',
       'video_length', 'url', 'message', 'link', 'final_link', 'image_text',
       'link_text', 'description', 'sponsor_id', 'sponsor_name',
       'sponsor_category'],
      dtype='object')

In [79]:
text_columns = set(['message', 'image_text','link_text','description'])
for col in text_columns:
    post_data[col] = post_data[col].str.lower()
    post_data[col] = post_data[col].str.replace('[^\w\s]','')

In [84]:
# Extracting NERs out of the messages and other text bodies
def get_entities(col, entities):
    null_count = 0
    for i in tqdm(range(len(post_data))):
        try:
            doc = nlp(post_data[col][i])
            entities.append([(ent.text, ent.label_) for ent in doc.ents])
        except (TypeError, ValueError):
            null_count += 1
    
    print(f"Number of null values in {col}: {null_count}")
    return entities
    
        

entities = []
for text_col in text_columns:
    entities = get_entities(text_col, entities)

entities

 53%|█████▎    | 4723/8983 [02:25<02:10, 32.52it/s]


KeyboardInterrupt: 